<a href="https://colab.research.google.com/github/kangwonlee/nmisp/blob/lecture-idea/50_ode/10_Forward_Euler.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


In [ ]:
# This cell is for the Google Colaboratory
# https://stackoverflow.com/a/63519730
if 'google.colab' in str(get_ipython()):
  path_py = '/content/nmisp_py'

  import os
  if not os.path.exists(path_py):
    import subprocess
    subprocess.run(
        ('git', 'clone', 'https://github.com/kangwonlee/nmisp_py')
    )
  assert os.path.exists(path_py)

  import sys
  sys.path.insert(0, path_py)



In [ ]:
# 그래프, 수학 기능 추가
# Add graph and math features
import pylab as py
import numpy as np
import numpy.linalg as nl



# 상미분방정식을 위한 오일러법<br>Euler Method for Ordinary Differntial Equation



[![Euler's method | Differential equations| AP Calculus BC | Khan Academy](https://i.ytimg.com/vi/q87L9R9v274/hqdefault.jpg)](https://www.youtube.com/watch?v=q87L9R9v274)



## 여러 $(t, x)$ 지점에서의 기울기<br>Slopes at $(t, x)$ points



다시 한번 주어진 미분 방정식을 생각해 보자.<br>Let's think about the first order differential equation again.



$$
\left\{
    \begin{align}
        a_0 \frac{d}{dt}x(t)+a_1 x(t)&=0 \\
        x(0)&=x_0 \\
    \end{align}
\right.
$$



미분항을 남기고 나머지를 등호의 오른쪽으로 옮겨 보자.<br>Let's move terms except the differential to the right side of the equal sign.



$$
a_0 \frac{d}{dt}x(t)=-a_1 x(t)
$$



양변을 $a_0$로 나누어 보자.<br>Let's divide both sides with $a_0$.



$$
\frac{d}{dt}x(t)=-\frac{a_1}{a_0} x(t)
$$



이 식의 의미를 한번 생각해 보자.<br>Let's think about the meaning of this equation.



위 미분방정식을 만족시키는 어떤 함수 $x(t)$의 $t=t_i$, $x=x_j$ 점에서의 $t$에 대한 기울기는 다음과 같을 것이라는 의미이다.<br>
This equation indicates that a function $x(t)$ satisfying the differential equation above would have a slope as follows at a point of $t=t_i$ and $x=x_j$.



$$
\left.\frac{d}{dt}x\right|_{\left(t, x\right)=\left(t_i,x_j\right)}=-\frac{a_1}{a_0} x_j
$$



이런식으로 $t$의 변화에 따른 $x$의 기울기 $\frac{d}{dt}x(t)$ 를 모든 $(t, x)$ 점에서 구할 수 있다.<br>
In this way, we can find all the $\frac{d}{dx}x(t)$, slopes of $x$ with respect to the change of $t$ at all $(t, x)$ points.



## 방향장 기울기 시각화<br>Visualizing the slopes in the [Direction Field](https://en.wikipedia.org/wiki/Direction_field)



다음 예를 생각해 보자.<br>
Let's think about an example as follows.



$$
\left\{
    \begin{align}
        2 \frac{d}{dt}x(t)+ x(t)&=0 \\
        x(0)&=x_0 \\
    \end{align}
\right.
$$



기울기를 계산하는 파이썬 함수를 생각해 보자.<br>Let's think about a python function calculating the slope.



In [ ]:
a_0, a_1 = 2.0, 1.0

def dx_dt(t, x):
    return - a_1 * x / a_0



예를 들어 $0 \le t \le 10$, $-6 \le x \le 6$ 인 영역에서 기울기를 그려 보자.<br>
Let's plot slopes within the region of $0 \le t \le 10$ and $-6 \le x \le 6$.



In [ ]:
t_slope = py.linspace(0, 10)
x_slope = py.linspace(-6, 6)



In [ ]:
import ode_plot

ode_plot.ode_slope_1state(dx_dt, x_slope, t_slope)
py.savefig('slopes_t_x.svg')



 위와 같이 모든 점에서의 기울기의 그림을 **방향장** 또는 **기울기장** 이라고 한다.<br>
 As above, a plot of the slopes of all points is called a [**Direction Field**](https://en.wikipedia.org/wiki/Direction_field) or **Slope Field**
 



엄밀해를 겹쳐 그려 보자<br>Let's overlap the curve of the exact solution.



$$x(t)=x_0 e^{-\frac{a_1}{a_0} t}$$



In [ ]:
import ode_plot


# Plot direction field
# 방향장 표시
ode_plot.ode_slope_1state(dx_dt, x_slope, t_slope)

def exact(t, x_0=4.5):
    return x_0 * py.exp((-a_1 / a_0) * t)

x_0 = 4.5
py.plot(t_slope, exact(t_slope, x_0), label=f"$x_0 = {x_0}$")

x_0 = 2.0
py.plot(t_slope, exact(t_slope, x_0), label=f"$x_0 = {x_0}$")

x_0 = -2.0
py.plot(t_slope, exact(t_slope, x_0), label=f"$x_0 = {x_0}$")

py.legend(loc=0)

py.savefig('slopes_t_x_exact.svg')



$t=0$에서의 $x$의 초기값에 따라 엄밀해가 달라질 수 있음을 기억하자.<br>Let's remember that the exact solution may vary depending on the initial value of $x$ at $t=0$.



이것을 이용해서 미분방정식의 해 곡선을 구해볼 수 있을까?<br>Using this, can we find solution curves of a differential equation?



## 오일러법<br>Euler Method



$\left(t,x\right) = \left(0, x_0\right)$ 에서의 $x(t)$의 기울기를 생각해 보자.<br>
Let's think about the slope of $x(t)$ at $\left(t,x\right) = \left(0, x_0\right)$.



$$
\left.\frac{d}{dt}x\right|_{\left(t, x\right)=\left(0,x_0\right)}=-\frac{a_1}{a_0} x_0=s_0
$$



그렇다면, $(0, x_0)$점을 지나고 기울기가 $s_0=-\frac{a_1}{a_0} x_0$ 인 직선을 생각할 수 있다.<br>
Then, we can think about a line passing through $(0, x_0)$ with the slope of $s_0=-\frac{a_1}{a_0} x_0$.



$$
x=-\frac{a_1}{a_0} x_0 \left(t -  0 \right) + x_0=s_0\left(t -  0 \right) + x_0
$$



이 직선을 따라 $t_1$ 까지 전진해 보자.<br>
Following this line, let's move forward to $t_1$.



$$
t_1=t_0 + \Delta t = 0 + 0.5
$$



In [ ]:
# Initial point
t_0 = 0
x_0 = 4.5

# time step
delta_t = 0.5



In [ ]:
# Slope at the initial point (t_0, x_0) point
s_0 = dx_dt(t_0, x_0)

# Straight line to next time step
t_0_array = py.linspace(0, delta_t)
x_0_array = s_0 * (t_0_array - t_0) + x_0

# (t_1, x_1) point
t_1, x_1 = t_0_array[-1], x_0_array[-1]



In [ ]:
ode_plot.indicate_initial_point(t_0, x_0)
ode_plot.plot_one_step(t_0_array, x_0_array, 0)
ode_plot.format_incremental_plot()



$(t_1, x_1)$ 에서의 기울기 $s_1$ 과 그러한 기울기를 가지고 $\left(t_1, x_1\right)$ 을 지나는 직선은 다음과 같다.<br>
The slope $s_1$ at $\left(t_1, x_1\right)$ and another line with such slope and passing $\left(t_1, x_1\right)$ would be as follows.



$$
\begin{align}
\left.\frac{d}{dt}x\right|_{\left(t, x\right) = \left(t_1,x_1\right)} &=-\frac{a_1}{a_0} x_1=s_1 \\
x & = s_1\left(t - t_1 \right) + x_1
\end{align}
$$



In [ ]:
# Slope at (t_1, x_1) point
s_1 = dx_dt(t_1, x_1)

# Straight line to next time step
t_1_array = py.linspace(t_1, t_1 + delta_t)
x_1_array = s_1 * (t_1_array - t_1) + x_1

# (t_2, x_2) point
t_2, x_2 = t_1_array[-1], x_1_array[-1]



In [ ]:
# Indicate the line from (t_0, x_0) with slope s_0
ode_plot.indicate_initial_point(t_0, x_0)
ode_plot.plot_one_step(t_0_array, x_0_array, 0)

# Indicate the line from (t_1, x_1) with slope s_1
ode_plot.plot_one_step(t_1_array, x_1_array, 1)

ode_plot.format_incremental_plot()



엄밀해와 비교해 보자.<br>Let's compare with the exact solution.



In [ ]:
# https://stackoverflow.com/a/9236970
t_array = py.concatenate([t_0_array.tolist(), t_1_array.tolist()], axis=None)
exact = ode_plot.ExactPlotterFirstOrderODE(t_array)



In [ ]:
# Indicate the line segments
ode_plot.indicate_initial_point(t_0, x_0)
ode_plot.plot_one_step(t_0_array, x_0_array, 0)
ode_plot.plot_one_step(t_1_array, x_1_array, 1)

# plot exact solution
exact.plot()

ode_plot.format_incremental_plot()



$t$값이 커 짐에 따라, 엄밀해와 $x_1$의 오차 보다 엄밀해와 $x_2$ 사이의 오차가 커지는 것을 볼 수 있다.<br>
As $t$ increases, the error between $x_2$ and the exact solution is larger than the error between $x_1$ and the exact solution.



## 함수로 변환<br>Convert to a function



사용상 편리를 위해 함수를 만들어 보자.<br>To make it easier to use, let's make it a function.



In [ ]:
def euler(f, t_array, x_0):
    '''
    Forward Euler Method
    전진 오일러법

    Parameters
    매개변수
    ----------
    f : callable 함수

        The calling signature is f(t, x)
        f(t, x) 와 같이 호출

        Returns slope at (t, x)
        (t, x) 점에서의 기울기를 반환

    t_array : array_like 배열과 비슷한 변수

        Collection of t at which we want to know x
        x를 구하고자 하는 시각의 모음

    x_0 : operatable 숫자 또는 배열

        Initial value of x
        x의 초기값

    Returns
    반환값
    -------
    time_list : list of t_i's 시간의 리스트
    result_list : list of x_i's x의 리스트
    '''

    # initialize results
    # 결과를 초기화
    time_list = [t_array[0]]
    result_list = [x_0]

    # initialize x
    # x를 초기화
    x_i = x_0

    # time step loop
    # 매 시각을 훑는 반복문
    for k, t_i in enumerate(t_array[:-1]):
        # time step interval
        # 시간 간격
        delta_t = t_array[k+1] - t_array[k]

        # slope at (t_i, x_i)
        # (t_i, x_i)에서의 기울기
        s_i = f(t_i, x_i)

        # x[i + 1]
        # i+1 단계의 x
        x_i_plus_1 = x_i + s_i * delta_t

        # append i + 1st t & x to the results
        # i + 1 번째 t와 x 를 결과에 추가
        time_list.append(t_array[k+1])
        result_list.append(x_i_plus_1)

        # x_i for the next step
        # 다음 단계를 위한 x_i
        x_i = x_i_plus_1
    # end of time step loop
    # 반복문 끝

    # return the results
    # 결과를 반환
    return time_list, result_list



다시 그려 보자.<br>Let's plot again.



In [ ]:
# Time step interval
delta_t = 0.5

# Time array
t_sec_array = np.arange(0, 1 + delta_t*0.5, delta_t)

# Initial state
x_0 = 4.5

# *** new function ***
t_euler_out, x_euler_out = euler(dx_dt, t_sec_array, x_0)
# *** new function ***

py.plot(t_euler_out, x_euler_out, 'o-')

for k in range(len(t_euler_out)):
    ode_plot.text_xy_k(t_euler_out[k], x_euler_out[k], k)

# Indicate the exact solution
exact.plot()

ode_plot.format_incremental_plot()



## $\Delta t$ 간격의 영향<br>Influence of $\Delta t$ interval



오차를 줄일 수 있는 좋은 방법이 없을까?<br>Is there a good way to reduce the error?



$\Delta t=0.5$ 를 $\Delta t=0.1$로 한번 줄여 보자.<br>Let's make $\Delta t=0.5$ to $\Delta t=0.1$.



In [ ]:
# Time step interval & Time array
delta_t = 0.1
t_sec_array = np.arange(0, 1 + delta_t*0.5, delta_t)

# Initial state
x_0 = 4.5

# *** new function ***
t_euler_out_01, x_euler_out_01 = euler(dx_dt, t_sec_array, x_0)



In [ ]:
py.plot(t_euler_out, x_euler_out, 'o-', label='$\Delta t=0.5$')
py.plot(t_euler_out_01, x_euler_out_01, '.-', label='$\Delta t=0.1$')

# Indicate the exact solution
exact.plot()

ode_plot.format_incremental_plot()



$\Delta t$ 간격을 줄이면 오차에 어떤 영향을 미쳤는가?<br>How did reducing $\Delta t$ interval influence the error?



### 근사해와 방향장<br>Approximate solutions and direction fields



해를 방향장과 겹쳐 그려 보자.<br>
Let's overlap the solutions and the direction field.



$t$와 $x$의 범위<br>
Ranges of $t$ and $x$



In [ ]:
t_slopes = py.linspace(0, 6)
x_slopes = py.linspace(0, 6)



초기값<br>Initial value<br>
$x(t_0)$



In [ ]:
x_0 = 4.5



$
\Delta t = 0.5 
$ (sec)



In [ ]:
delta_t_05 = 0.5
t_05_sec = np.arange(t_slopes[0], t_slopes[-1] + delta_t_05*0.5, delta_t_05)
t_out_05, x_out_05 = euler(dx_dt, t_05_sec, x_0)



$
\Delta t = 0.1 
$ (sec)



In [ ]:
delta_t_01 = 0.1
t_01_sec = np.arange(t_slopes[0], t_slopes[-1] + delta_t_01*0.5, delta_t_01)
t_out_01, x_out_01 = euler(dx_dt, t_01_sec, x_0)



이제 그려 보자.<br>Now let's plot.



In [ ]:
# Slopes at each (t, x) points
ode_plot.ode_slope_1state(dx_dt, x_slopes, t_slopes)

py.plot(t_out_05, x_out_05, 'o-', label='$\Delta t=0.5$')
py.plot(t_out_01, x_out_01, 'o-', label='$\Delta t=0.1$')

# plot exact solution
exact = ode_plot.ExactPlotterFirstOrderODE(t_slopes)
exact.plot()

# Aspect ratio
py.axis('equal')

# xy limits
py.xlim(left=t_slopes[0], right=t_slopes[-1])
py.ylim(bottom=x_slopes[0], top=x_slopes[-1])

py.legend(loc=1, fontsize='xx-large');



$i$번째 점과 $i+1$번째 점 사이에서 $\frac{d}{dt}x(t)$는 계속 변화하고 있으나, 오일러법은 해당 구간에서의 기울기가 $\frac{d}{dt}x(t_i)$인 것으로 가정한다.<br>
Between the $i$th and $i+1$st points, $\frac{d}{dt}x(t)$ continuously changes.  However, the Euler Method assumes that the slope is fixed at $\frac{d}{dt}x(t_1)$ within the interval.



그렇다면, $t_i \le t \le t_{i+1}$ 사이에서 대표적인 $\frac{d}{dt}x$ 값은 어떤 값이 좋을 것인가?<br>
If so, within $t_i \le t \le t_{i+1}$, which value of $\frac{d}{dt}x$ would be representative?



## Scipy



과학기술 계산 라이브러리인 사이파이 `scipy` 의 `scipy.integrate` 를 통해서 다수의 상미분 방정식 솔버 solver 를 제공한다.<br>
As a scientific computation library, `scipy` has ODE solvers in `scipy.integrate`.



In [ ]:
import scipy.integrate as si



In [ ]:
sol = si.solve_ivp(dx_dt, (t_out_01[0], t_out_01[-1]), [x_0], t_eval=t_out_01)



엄밀해, 오일러법 결과와 비교해보자.<br>
Let's compare with the exact solution and the result of the Euler's method.



In [ ]:
py.plot(sol.t, sol.y[0, :], 'o', label='solve_ivp')
py.plot(t_out_01, x_out_01, '.-', label='$\Delta t=0.1$')

# plot exact solution
exact = ode_plot.ExactPlotterFirstOrderODE(t_slopes)
exact.plot()
py.grid(True)
py.xlabel('t(sec)')
py.ylabel('y(t)')
py.legend(loc=0);



판다스 `pandas` 를 이용해 표 형태로도 살펴볼 수 있을 것이다.<br>
The `pandas`would enable observing in a table form.



In [ ]:
import pandas as pd


df = pd.DataFrame(
    data={
        'euler':x_out_01,
        'solve_ivp':sol.y[0, :],
        'exact':exact.exact(py.array(t_out_01))
    },
    index=pd.Series(t_out_01, name='t(sec)'),
    columns=['exact', 'euler', 'solve_ivp']
)



열 연산으로 엄밀해에 대한 오차를 구해보자.<br>
Let's calculate the error against the exact solution.



In [ ]:
df['euler_error'] = df.euler - df.exact
df['solve_ivp_error'] = df.solve_ivp - df.exact



표 형태<br>Table form



In [ ]:
pd.set_option('display.max_rows', 10)
df



각종 통계<br>Statistics



In [ ]:
df.describe()



이 경우, 오일러법의 오차에 대한 의견은?<br>
In this case, what do you think about the error of the Euler method?



In [ ]:
import numpy.linalg as nl


nl.norm(df.euler_error), nl.norm(df.solve_ivp_error), 



## 연습 문제<br>Exercise


도전과제 1: 다음 미분방정식의 해곡선을 전진 오일러법으로 구하시오.<br>
Try This 1: Find a solution curve of the following differential equation using the forward Euler method.



$$
\begin{cases}
\begin{align}
a_0\frac{d}{dt}x(t) + a_1 x(t) &= 0 \\
x(0)&=x_0
\end{align}
\end{cases}
$$


도전과제 2: 다음 미분방정식의 해곡선을 전진 오일러법으로 구하시오.<br>
Try This 2: Find a solution curve of the following differential equation using the forward Euler method.



$$
\begin{cases}
\begin{align}
a_0 \frac{d}{dt}x(t) + a_1 x(t) &= b_0 t \\
x(0)&=0
\end{align}
\end{cases}
$$


도전과제 3: 다음 미분방정식의 해곡선을 전진 오일러법으로 구하시오.<br>
Try This 3: Find a solution curve of the following differential equation using the forward Euler method.



$$
\begin{cases}
\begin{align}
a_0 \frac{d}{dt}x(t) + a_1 x(t) &= b_0 sin(t) \\
x(0)&=x_0
\end{align}
\end{cases}
$$


도전과제 4: 다음 미분방정식의 해곡선을 전진 오일러법으로 구하시오.<br>
Try This 4: Find a solution curve of the following differential equation using the forward Euler method. [[link](https://en.wikipedia.org/wiki/Bernoulli_differential_equation)]<br>
$1 \le x \le 10$



$$
\begin{cases}
\begin{align}
\frac{d}{dx}y(x) - \frac{2}{x}y(x) &= -x^2y^2 \\
y(1)&=1
\end{align}
\end{cases}
$$


## 예상 자료형<br>Type hints



References : [[1](https://docs.python.org/3/library/typing.html)], [[2](https://stackoverflow.com/a/54610845)]



타입 힌트는 변수의 예상 자료형을 예시하여 (개발자 또는) 개발용 소프트웨어 등을 돕기 위한 것이다.<br>
Type hints are to present expected data types to help (developers or) development software.



In [ ]:
def dx_dt_type_hints(t:float, x:float) -> float:
    return - a_1 / a_0 * x



예를 들어 위 함수는 실수형 매개변수 `t`, 실수형 매개변수 `x` 를 받아들여 계산한 결과를 실수형으로 반환할 것이라는 의미이다.<br>
For example, the function above would accept two `float` arguments of `t` and `x` to return the calculated result in  `float`.



매개변수 `t` 가 실수형 또는 정수형으로 예상된다면, 다음과 같이  `typing` 모듈의 `Union`으로 표시할 수 있다.<br>
If the argument `t`is expected to be `float` or `int`, we can indicate as follows using `Union` of the `typing` module.



In [ ]:
from typing import Union


def dx_dt_type_hints(t:Union[float, int], x:float) -> float:
    return - a_1 / a_0 * x



`euler( )`와 같이 함수 또는 list를 받아들이는 경우는 다음과 같이 표시할 수 있을 것이다.<br>
If arguments are a function or a list as `euler( )`, following cell would be possible.



In [ ]:
from typing import Callable, List, Tuple, Union


import numpy as np

Scalar = Union[float, int]
Time = Scalar
TimeList = Union[List[Time], Tuple[Time]]
State = Union[Scalar, List[Scalar], Tuple[Scalar], np.ndarray]
StateList = Union[List[State], Tuple[State]]
SlopeFunction = Callable[[Time, State], State]


def euler(f:SlopeFunction, t_array:TimeList, x_0:State) -> Tuple[TimeList, StateList]:

    time_list = [t_array[0]]
    result_list = [x_0]

    x_i = x_0

    for k, t_i in enumerate(t_array[:-1]):
        # time step
        delta_t = t_array[k+1] - t_array[k]

        # slope
        s_i = f(t_i, x_i)

        # x[i + 1]
        x_i_plus_1 = x_i + s_i * delta_t

        time_list.append(t_array[k+1])
        result_list.append(x_i_plus_1)
        
        x_i = x_i_plus_1

    return time_list, result_list



## 중첩함수와 클로져<br>Nested function and Closure



내장 함수는 다른 함수 안에 정의된 함수이다.<br>A nested function is a function defined another function.



In [ ]:
def function_with_a_nested_function(t_list, x_list, a=0.5, b=2):

    def nested_function(t_i, x_i):

        return a * t_i + b * x_i

    y_list = []

    for t_j, x_j in zip(t_list, x_list):
        y_list.append(nested_function(t_j, x_j))

    return y_list



In [ ]:
t_list = [0, 1, 2]
x_list = [0.1, 0.2, 0.3]
function_with_a_nested_function(t_list, x_list)



**클로져**는 이러한 *내장 함수가 함수의 반환값*인 경우라고 할 수 있다.<br>
A **closure** can be a nested function returned from a function.



In [ ]:
def function_returning_a_closure(a=0.5, b=2):

    def this_will_be_a_closure(t_i, x_i):

        return a * t_i + b * x_i

    return this_will_be_a_closure # no ( )



In [ ]:
this_is_a_closure = function_returning_a_closure()

t_list_closure = [0, 1, 2]
x_list_closure = [0.1, 0.2, 0.3]
y_list_closure = []

for t_j, x_j in zip(t_list_closure, x_list_closure):
    y_list_closure.append(this_is_a_closure(t_j, x_j))

y_list_closure



예를 들어 미분방정식의 기울기를 계산하는 함수를 closure 로 생성하는 것도 가능할 것이다.<br>
For example, we may think about a function returning a closure calculating the slope for the differential equation.



$$
\frac{d}{dt}x(t)=-\frac{a_1}{a_0}x(t)
$$



In [ ]:
def get_slope_function_LODE1(a0, a1):

    '''
    Make a function calculating the slope
    기울기를 계산하는 함수를 만들어주는 함수
    '''

    # Coefficient to calculate the slope
    # 기울기를 계산하기 위해 필요한 계수
    minus_a1_over_a0 = -a1 / a0
    # This variable would survive even after this function returns
    # get_slope_function_LODE1() 함수가 반환된 후에도 이 변수는 수명을 유지할 것이다.

    # Following nested function would calculate slope at (t, x)
    # 아래 내부 함수는 (t, x) 에서의 기울기를 계산할 것이다.
    def slope_LODE1(t, x):
        return minus_a1_over_a0 * x

    # The nested function is the return value
    # 내부 함수가 반환값이 된다.
    return slope_LODE1



We can make the slope function as follows.<br>
아래와 같이 기울기 함수를 만들 수 있다.



In [ ]:
dx_dt_closure = get_slope_function_LODE1(a_0, a_1)



Let's apply the original function and the closure to the `euler()`.<br>
원래 함수와 클로져를 `euler()`에 적용해 보자.



In [ ]:
delta_t_01 = 0.1
t_01_sec = np.arange(t_slopes[0], t_slopes[-1] + delta_t_01*0.5, delta_t_01)

# Original function
# 원래의 함수
(t_out_01, x_out_01) = euler(dx_dt, t_01_sec, x_0)

# Closure
# 클로져
(t_out_01_closure, x_out_01_closure) = euler(dx_dt_closure, t_01_sec, x_0)



In [ ]:
import numpy as np
import numpy.testing as nt



If the tests below pass, results using the closure is close to the original function.<br>
아래 테스트가 통과된다면, 클로져를 사용한 결과와 원래 함수의 결과는 서로 가까운 것이다.



In [ ]:
nt.assert_allclose(
  np.array(t_out_01),
  np.array(t_out_01_closure)
)

nt.assert_allclose(
  np.array(x_out_01),
  np.array(x_out_01_closure)
)



Following cell compares return values from functions `dx_dt()` and `dx_dt_closure()` using `t_out_01` and `x_out_01` results.<br>
아래 셀은 `t_out_01` 과 `x_out_01` 결과를 이용하여 함수 `dx_dt()` 와 `dx_dt_closure()`의 반환값을 비교할 것이다.



In [ ]:
nt.assert_allclose(
  dx_dt(np.array(t_out_01), np.array(x_out_01)),
  dx_dt_closure(np.array(t_out_01), np.array(x_out_01))
)



## Final Bell<br>마지막 종



In [ ]:
# stackoverfow.com/a/24634221
import os
os.system("printf '\a'");

